# 3.3

## 3.3.1

In [162]:
# Start with an input sequence
import torch
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your
     [0.55, 0.87, 0.66], # journey
     [0.57, 0.85, 0.64], # starts
     [0.22, 0.58, 0.33], # with
     [0.77, 0.25, 0.10], # one
     [0.05, 0.80, 0.55]] # step
)

In [163]:
# We calculate the intermediate attention scores between the query token "journey" and each input token
query = inputs[1]
attention_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attention_scores_2[i] = torch.dot(x_i, query)
print(attention_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [164]:
# Normalize each of the attention scores
attention_weights_2_temp = attention_scores_2 / attention_scores_2.sum()
print("Attention weights:", attention_weights_2_temp)
print("Sum:", attention_weights_2_temp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [165]:
# Implement softmax
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim = 0)
attention_weights_2_naive = softmax_naive(attention_scores_2)
print("Attention weights:", attention_weights_2_naive)
print("Sum:", attention_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [166]:
# Use PyTorch implementation of softmax
attention_weights_2 = torch.softmax(attention_scores_2, dim = 0)
print("Attention weights:", attention_weights_2)
print("Sum:", attention_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [167]:
# Calculating the context vector
# Multiplying the embedded input tokens x^i with the corresponding attnetion weights and then summing the resulting vectors
query = inputs[1]
context_vector_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vector_2 += attention_weights_2[i] * x_i
print(context_vector_2)

tensor([0.4419, 0.6515, 0.5683])


## 3.3.2

In [168]:
# Compute all context vectors
attention_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attention_scores[i, j] = torch.dot(x_i, x_j)
print(attention_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [169]:
# Using matrix multiplication
attention_scores = inputs @ inputs.T
print(attention_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [170]:
# Get attention weights through normalizing attention scores
attention_weights = torch.softmax(attention_scores, dim = -1)
print(attention_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [171]:
# Verify summing to 1
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)
print("All row sums:", attention_weights.sum(dim = -1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [172]:
# Compute all context vectors with matrix multiplication
all_context_vectors = attention_weights @ inputs
print(all_context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [173]:
# Check for correctness
print("Previous 2nd context vector:", context_vector_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


# 3.4

## 3.4.1

In [174]:
# Introducing attention weights
x_2 = inputs[1]
d_in = inputs.shape[1] # Input embedding size
d_out = 2 # Output embedding size

In [175]:
# Initialize the three weight matrices
torch.manual_seed(123)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)

In [176]:
# Get Query, Key, and Value vectors
key_2 = x_2 @ W_key
query_2 = x_2 @ W_query
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4433, 1.1419])


In [177]:
# Obtain all keys and values
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [178]:
# Compute attention score
attention_score_22 = query_2.dot(key_2)
print(attention_score_22)

tensor(1.8524)


In [179]:
# Compute all attention scores
attention_scores_2 = query_2 @ keys.T
print(attention_scores_2)

tensor([1.3545, 1.8524, 1.8284, 1.0167, 0.8819, 1.3165])


In [180]:
# Go grom attention scores to attention weights
d_k = keys.shape[-1]
print("d_k:", d_k)
attention_weights_2 = torch.softmax(attention_scores_2 / d_k ** 0.5, dim = -1)
print(attention_weights_2)

d_k: 2
tensor([0.1588, 0.2258, 0.2220, 0.1251, 0.1137, 0.1546])


In [181]:
# Compute context vector
context_vector_2 = attention_weights_2 @ values
print(context_vector_2)

tensor([0.3015, 0.8132])


## 3.4.2

In [182]:
# Implementing a compact self-attention class
class SelfAttention_v1(torch.nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = torch.nn.Parameter(torch.rand(d_in, d_out))
    
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attention_scores = queries @ keys.T
        attention_weights = torch.softmax(attention_scores / keys.shape[-1] ** 0.5, dim = -1)
        context_vector = attention_weights @ values
        return context_vector

In [183]:
# Using this class
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [184]:
# Using nn.Linear
class SelfAttention_v2(torch.nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.W_query = torch.nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias = qkv_bias)
    
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attention_scores = queries @ keys.T
        attention_weights = torch.softmax(attention_scores / keys.shape[-1] ** 0.5, dim = -1)
        context_vector = attention_weights @ values
        return context_vector

In [185]:
# Use this new class
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


### Exercise 3.1

In [186]:
# Compare two self-attentions
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)
torch.manual_seed(789)
print(sa_v1(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
